# Aggregations

To translate into rough SQL terms:

>SELECT COUNT(color) 

>FROM table

>GROUP BY color 


- COUNT(color) is equivalent to a metric.
- GROUP BY color is equivalent to a bucket.


## 1. Metric
Aggregations that keep track and compute metrics over a set of documents.
## 2. Matrix
Operate on multiple fields and produce a matrix result based on the values extracted from the requested document fields. No support of scripting.
## 3. Pipeline
Aggregations that aggregate the output of other aggregations and their associated metrics


## 4. Bucketing
Each bucket is associated with a key and a document criterion. When the aggregation is executed, all the buckets criteria are evaluated on every document in the context and when a criterion matches, the document is considered to "fall in" the relevant bucket. By the end of the aggregation process, we’ll end up with a list of buckets - each one with a set of documents that "belong" to it.

>Bucketing aggregations can have sub-aggregations (bucketing or metric). The sub-aggregations will be computed for the buckets which their parent aggregation generates. 


## Examples of Bucket Aggregation
- Date Histogram Aggregation
- Date Range Aggregation
- Filter(s) Aggregation
- Geo Distance Aggregation
- Histogram Aggregation
- Missing Aggregation
- Nested Aggregation
- Range Aggregation
- Reverse nested Aggregation
- Sampler Aggregation
- Significant Terms Aggregation
- Terms Aggregation

In [23]:
import requests
from faker import Faker
from faker import Factory
import ujson as json
import pandas as pd
import requests
import certifi
from numpy import random 
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch_dsl import Search, DocType, Date, Integer, Keyword, Text
from datetime import datetime
from elasticsearch_dsl.connections import connections
import pandas as pd
import requests
from random import randint
from datetime import datetime, timedelta
import math
import numpy as np
from pandas.io.json import json_normalize
from pandasticsearch import DataFrame
from pandasticsearch import Select

ES_HOST = 'http://ec2-34-205-15-150.compute-1.amazonaws.com:9200'
INDEX = "eda_new"
DOC_TYPE="user"
es = Elasticsearch(ES_HOST)



## Aggregation Format

```
"aggregations" : {
    "<aggregation_name>" : {
        "<aggregation_type>" : {
            <aggregation_body>
        }
        [,"meta" : {  [<meta_data_body>] } ]?
        [,"aggregations" : { [<sub_aggregation>]+ } ]?
    }
    [,"<aggregation_name_2>" : { ... } ]*
}
```


### Metrics Aggregation
Some aggregations output a single numeric metric (e.g. avg) and are called single-value numeric metrics aggregation, others generate multiple metrics (e.g. stats) and are called multi-value numeric metrics aggregation

In [24]:
query = {
            "from" : 0, 
            "size" : 50,
            "query": {
                        "match_all": {}
                     }
            }
result_dict = es.search(index=INDEX, body=query)
df = Select.from_dict(result_dict).to_pandas()
df = df.set_index(['_id'])
df.drop(['_index', '_score', '_type'],inplace=True,axis=1,errors='ignore')


In [68]:
df.head()

,Avg Order Value,City,days_since_last_purchase,discount_percentage,email_unsubscribe,f_score,lifecycle,m_score,name,num_of_orders,r_score,revenue,rfm_score,segments,total_discount_revenue
_id,,,,,,,,,,,,,,,
0ce81a1e-f3c2-02d4-8fd1-49307ad26c1b,1128.568182,Lucknow,52,29,False,8,Loyal (Sleeping),8,Michael Goodman,44,5,49657,21,[High Discount Seeker],14400.53
4d88c2e9-fb30-2fda-4b23-807b08457cf0,3375.538462,Chennai,29,23,False,5,Loyal (Sleeping),8,Tony Adams,13,7,43882,20,[High Cart Value],10092.86
fad0c0a5-8ec6-721e-8180-d902cf1fa86d,674.220000,Noida,24,20,False,8,Loyal (Sleeping),8,Eric Clark,50,7,33711,23,[NCR],6742.20
c1e9d162-7349-ee0f-556d-264aca508b47,3249.153846,Ahmedabad,45,10,False,5,Loyal (Sleeping),8,Daniel Thomas,13,5,42239,18,[High Cart Value],4223.90
18a3b227-a9e6-412d-25c9-771966d9cd02,1089.304348,Chennai,1,23,False,6,Loyal (Active),8,Samuel Miller,23,9,25054,23,[],5762.42


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 0ce81a1e-f3c2-02d4-8fd1-49307ad26c1b to cb218c15-d6ab-46da-4de2-3f9b107f921e
Data columns (total 15 columns):
Avg Order Value             50 non-null float64
City                        50 non-null object
days_since_last_purchase    50 non-null int64
discount_percentage         50 non-null int64
email_unsubscribe           50 non-null bool
f_score                     50 non-null int64
lifecycle                   50 non-null object
m_score                     50 non-null int64
name                        50 non-null object
num_of_orders               50 non-null int64
r_score                     50 non-null int64
revenue                     50 non-null int64
rfm_score                   50 non-null int64
segments                    50 non-null object
total_discount_revenue      50 non-null float64
dtypes: bool(1), float64(2), int64(8), object(4)
memory usage: 5.9+ KB


In [45]:
#################
# AVG 
################$
agg_name = "avg_revenue"
agg_type = "avg"
field_name = "revenue"

query = {
     "size": 0,
     "aggs" : {
        agg_name : { agg_type : { "field" : field_name } }
    }
}
res = es.search(index=INDEX, body=json.dumps(query))
print("{} for all customer is {}".format(agg_name, res['aggregations'][agg_name]['value']))

avg_revenue for all customer is 2840.2725987993995


#### Fielddata error
Fielddata is disabled on text fields by default.
```PUT /eda_xsmall/_mapping/City?update_all_types
{
"properties": {
"City": {
"type": "text",
"fielddata": true
}
}
}
```

In [44]:
#############################
## Cardinality Aggregation 
#############################
## A single-value metrics aggregation that calculates an 
## approximate count of distinct values. 

agg_name = "city_count"
agg_type = "cardinality"
field_name = "City"

query = {
     "size": 0,
     "aggs" : {
        agg_name : {
            agg_type : {
                "field" : field_name
            }
        }
    }
}
res = es.search(index=INDEX, body=json.dumps(query))
print("{} for all customer is {}".format(agg_name, res['aggregations'][agg_name]['value']))

city_count for all customer is 20


### Extended Stats Aggregation
A multi-value metrics aggregation that computes stats over numeric values extracted from the aggregated documents. These values can be extracted either from specific numeric fields in the documents, or be generated by a provided script.

```
{
    "aggs" : {
        agg_name : { "extended_stats" : { "field" : field_name } }
    }
}

```

> **Standard Deviation and Bounds require normality**

>The standard deviation and its bounds are displayed by default, but they are not always applicable to all data-sets. Your data must be normally distributed for the metrics to make sense. The statistics behind standard deviations assumes normally distributed data, so if your data is skewed heavily left or right, the value returned will be misleading.

In [43]:
#############################
## Extended Stats Aggregation 
#############################
agg_name = "stats_for_revenue"
agg_type = "extended_stats"
field_name = "revenue"

query = {
     "size": 0,
    "aggs" : {
        agg_name : {
            agg_type : {
                "field" : field_name
            }
        }
    }
}
res = es.search(index=INDEX, body=json.dumps(query))
for key, value in (res['aggregations'][agg_name]).items():
    print("{}: {}".format(key, value))

std_deviation_bounds: {'lower': -11967.630876011537, 'upper': 17648.176073610335}
min: 0.0
sum: 227108197.0
std_deviation: 7403.9517374054685
max: 99811.0
avg: 2840.2725987993995
sum_of_squares: 5028336555235.0
count: 79960
variance: 54818501.32982946


### Percentiles Aggregation & Ranks
A multi-value metrics aggregation that calculates one or more percentiles over numeric values extracted from the aggregated documents. 

In [52]:
#############################
## Percentiles Aggregation 
#############################
agg_name = "revenue_percentiles"
agg_type = "percentiles"
field_name = "revenue"

query =   {
    "size": 0,
    "aggs" : {
        agg_name : {
            agg_type : {
                "field" : field_name,
                "percents" : [10,20,30,40,50,60,70,80,90] 
            }
        }
    }
}

res = es.search(index=INDEX, body=query)

for key, value in res['aggregations'][agg_name]['values'].items():
        print(str(key) + ": " + str(value))
        
        

90.0: 6133.646676456798
60.0: 1101.6169794607754
20.0: 0.0
80.0: 2404.950590653953
10.0: 0.0
40.0: 682.1919102473319
30.0: 332.3090476990202
70.0: 1489.452100819937
50.0: 903.3986023022348


In [58]:
#############################
## Percentiles Rank 
#############################
agg_name = "Revenue Percentile Rank"
agg_type = "percentile_ranks"
field_name = "revenue"

query =   {
    "size": 0,
    "aggs" : {
        agg_name : {
            agg_type : {
                "field" : field_name,
                "values" : [1000,20000] 
            }
        }
    }
}

res = es.search(index=INDEX, body=query)

for key, value in res['aggregations'][agg_name]['values'].items():
        print(str(key) + ": " + str(value))
        

1000.0: 54.9574620552982
20000.0: 96.67190164683117


## Sum Aggregation
Metrics aggregation that sums up numeric values that are extracted from the aggregated documents. These values can be extracted either from specific numeric fields in the documents, or be generated by a provided script.



In [87]:
#############################
## SUM AGGREGATION
#############################
agg_name = "Avg. Noida_Revenue"
agg_type = "stats"
field_name = "revenue"
searchcity ="Surat"

query = {
    "size": 0,
    "query" : {
        "constant_score" : {
            "filter" : {
                "match" : { "City" : searchcity }
            }
        }
    },
    "aggs" : {
       agg_name  : { agg_type : { "field" : field_name } }
    }
}
res = es.search(index=INDEX, body=query)
print("Stats for Customers from {}". format(searchcity))
print("------------------------------")
for key, value in res['aggregations'][agg_name].items():
        print(str(key) + ": " + str(value))

Stats for Customers from Surat
------------------------------
avg: 2874.2712418300653
min: 0.0
sum: 11433851.0
count: 3978
max: 98725.0
